# Proyecto Final IH: Recomendador de Codewars|Github

In [1]:
# https://www.codewars.com/users/leaderboard

In [84]:
# imports 

import requests
from bs4 import BeautifulSoup


## Leaderboard
Top 500 de codewars

In [85]:
# URL

url = 'https://www.codewars.com/users/leaderboard'
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")

In [86]:
# usuarios de leaderboard

leaderboard = {e.text for e in soup.select('tr a')}
users = '\n'.join(leaderboard)


In [87]:
# Salvamos los nombres en un documento

with open('../output/500-leaders.txt', 'w') as f: 
    f.write(users) 

## Get Social

In [88]:
# get users in clan

# https://www.codewars.com/users/albertogcmr/following
# https://www.codewars.com/users/albertogcmr/followers
# https://www.codewars.com/users/albertogcmr/allies

def get_social(user): 
    res = set()
    for link in ['following', 'followers', 'allies']: 
        url = 'https://www.codewars.com/users/{}/{}'.format(user, link)
        html = requests.get(url).content

        soup = BeautifulSoup(html, "lxml")
        res = res.union({e.text for e in soup.select('table a')})
    return res
get_social('albertogcmr')

{'Girgetto',
 'J4CODE',
 'Raulvv',
 'Tommytrg',
 'VictorIrix',
 'VivianSolide',
 'bel',
 'boyander',
 'epangar',
 'evapanizo',
 'josh35767',
 'mc100s',
 'papuarza',
 'silvio.galli',
 'xFloki',
 'yacineh'}

## Get stats from user

In [90]:
res = ""
url = 'https://www.codewars.com/users/{}'.format('albertogcmr')
html = requests.get(url).content
soup = BeautifulSoup(html, "lxml")

In [91]:
# member since

member_since = soup.select('.stat-box div')[3].text
member_since = member_since.split(':')[-1]
member_since

'Oct 2018'

In [92]:
# last seen

last_seen = soup.select('.stat-box div')[4].text
last_seen = last_seen.split(':')[-1]
last_seen

'Dec 2018'

In [122]:
# github

github = soup.select('.stat-box div')[5].find_all('a', href=True)[0]['href']
github

'https://github.com/albertogcmr'

In [124]:
# linkedin

github = soup.select('.stat-box div')[5].find_all('a', href=True)[1]['href']
github

'https://www.linkedin.com/in/albertogarciacobo/'